In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
import sys
sys.path.append("..")
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

736

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+ weather_api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": current_description
                          })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

 Record 40 of Set 6 | simeykyne
Processing Record 41 of Set 6 | sentyabrskiy
City not found. Skipping...
Processing Record 42 of Set 6 | havre-saint-pierre
Processing Record 43 of Set 6 | kendari
Processing Record 44 of Set 6 | roswell
Processing Record 45 of Set 6 | rawannawi
City not found. Skipping...
Processing Record 46 of Set 6 | nieuw amsterdam
Processing Record 47 of Set 6 | ambulu
Processing Record 48 of Set 6 | bilibino
Processing Record 49 of Set 6 | carlisle
Processing Record 50 of Set 6 | flagstaff
Processing Record 1 of Set 7 | tarnow
Processing Record 2 of Set 7 | rochegda
Processing Record 3 of Set 7 | grand river south east
City not found. Skipping...
Processing Record 4 of Set 7 | clyde river
Processing Record 5 of Set 7 | palana
Processing Record 6 of Set 7 | changji
Processing Record 7 of Set 7 | alotau
City not found. Skipping...
Processing Record 8 of Set 7 | geraldton
Processing Record 9 of Set 7 | chagda
City not found. Skipping...
Processing Record 10 of Set 7 

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hofn,IS,64.2539,-15.2082,53.80,83,34,13.44,scattered clouds
1,Bluff,NZ,-46.6000,168.3333,45.39,85,88,4.45,overcast clouds
2,Tuktoyaktuk,CA,69.4541,-133.0374,53.60,87,90,12.66,overcast clouds
3,Ketchikan,US,55.3422,-131.6461,61.27,77,90,6.91,light rain
4,Rikitea,PF,-23.1203,-134.9692,71.78,61,100,26.08,overcast clouds
5,Ushuaia,AR,-54.8000,-68.3000,38.86,87,90,9.22,light intensity drizzle
6,Jamestown,US,42.0970,-79.2353,80.29,67,1,4.00,clear sky
7,Bethel,US,41.3712,-73.4140,85.19,80,75,1.01,broken clouds
8,Adrar,MR,20.5022,-10.0711,105.75,7,26,8.70,scattered clouds
9,Belaya Gora,RU,68.5333,146.4167,47.70,56,100,5.10,overcast clouds


In [8]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")